In [1]:
import json
import numpy as np
import pandas as pd
from pprint import pprint
from collections import defaultdict

In [2]:
vistext_test_captions = np.load("./vistext_test_id_to_captions.npy", allow_pickle=True).item()
matplotalt_captions = np.load("./vistext_test_id_to_matplotalt_captions.npy", allow_pickle=True).item()
gpt4v_L3_captions = np.load("./vistext_test_id_to_L3_gpt4_captions.npy", allow_pickle=True).item()
gpt4v_L3_alt_captions = np.load("./vistext_test_id_to_L3_gpt4_alt_captions.npy", allow_pickle=True).item()
gpt4v_L3_table_captions = np.load("./vistext_test_id_to_L3_gpt4_table_captions.npy", allow_pickle=True).item()
gpt4v_L3_table_alt_captions = np.load("./vistext_test_id_to_L3_gpt4_table_alt_captions.npy", allow_pickle=True).item()
gpt4v_L4_captions = np.load("./vistext_test_id_to_L4_gpt4_captions.npy", allow_pickle=True).item()
gpt4v_L4_alt_captions = np.load("./vistext_test_id_to_L4_gpt4_alt_captions.npy", allow_pickle=True).item()
gpt4v_L4_table_captions = np.load("./vistext_test_id_to_L4_gpt4_table_captions.npy", allow_pickle=True).item()

In [5]:
vistext_id_to_combined_captions = []
for image_id, caption_dict in vistext_test_captions.items():
    image_dict = {}
    image_dict["image_id"] = image_id
    image_dict["human"] = [hc["L1"] + " " + hc["L2L3"] for hc in caption_dict]
    if image_id in matplotalt_captions:
        image_dict["heuristic"] = [matplotalt_captions[image_id]]
    if image_id in gpt4v_L4_captions:
        image_dict["gpt-4-turbo-L4"] = [gpt4v_L4_captions[image_id]]
    if image_id in gpt4v_L4_alt_captions:
        image_dict["gpt-4-turbo-alt-L4"] = [gpt4v_L4_alt_captions[image_id]]
    if image_id in gpt4v_L4_table_captions:
        table_caption = gpt4v_L4_table_captions[image_id]
        table_idx = table_caption.lower().find("\n\ndata table:")
        if table_idx != -1:
            table_caption = table_caption[:table_idx]
        image_dict["gpt-4-turbo-table-L4"] = [table_caption]
    if image_id in gpt4v_L3_captions:
        image_dict["gpt-4-turbo-L3"] = [gpt4v_L3_captions[image_id]]
    if image_id in gpt4v_L3_alt_captions:
        image_dict["gpt-4-turbo-alt-L3"] = [gpt4v_L3_alt_captions[image_id]]
    if image_id in gpt4v_L3_table_captions:
        table_caption = gpt4v_L3_table_captions[image_id]
        table_idx = table_caption.lower().find("\n\ndata table:")
        if table_idx != -1:
            table_caption = table_caption[:table_idx]
        image_dict["gpt-4-turbo-table-L3"] = [table_caption]
    if image_id in gpt4v_L3_table_alt_captions:
        image_dict["gpt-4-turbo-table-alt-L3"] = [gpt4v_L3_table_alt_captions[image_id]]
    vistext_id_to_combined_captions.append(image_dict)

combined_captions_df = pd.DataFrame.from_dict(vistext_id_to_combined_captions)
combined_captions_df.to_json("./vistext_id_to_combined_captions.jsonl", orient='records', lines=True)

In [6]:
combined_captions_df = pd.read_json("./vistext_id_to_combined_captions.jsonl", orient='records', lines=True)
new_heuristic = []
def rh(row):
    new_heuristic.append([r.replace("1 variables are plotted for the data. ", "") for r in row["heuristic"]])
combined_captions_df.apply(rh, axis=1)
combined_captions_df["heuristic"] = new_heuristic
print(new_heuristic)
combined_captions_df.to_json("./vistext_id_to_combined_captions.jsonl", orient='records', lines=True)

[["A bar chart titled 'number of car registrations in europe in 2019 , by segment (in 1,000s)'. New registrations in thousands is plotted on the x-axis from 0 to 3000 using a linear scale and programming language is plotted on the y-axis from small to midsize using a categorical scale. The data has a minimum value of x=403.9 at y=midsize and a maximum value of x=2685 at y=small. The data strictly decrease to a min at y=midsize."], ["A line plot titled 'number of hospitals in germany from 2000 to 2017'. Year is plotted on the x-axis from 1998 to 2018 and number of hospitals is plotted on the y-axis from 0 to 4000, both using linear scales. The data has a minimum value of y=3084 at x=2016 and a maximum value of y=3635 at x=1999. The data generally decrease to a min at x=2016."], ["A bar chart titled 'number of mobile customers of vodafone in germany in the corporate years from 2008/09 to 2019/2020 (in millions)'. Year is plotted on the x-axis from 2019/20 to 2008/09 using a categorical s